# Submissions

<p style="text-align:justify;">
Nineteen participants submitted data that were approved, which included 41 T1 maps of the NIST/system phantom, and 56 brain T1 maps. It should be noted that these numbers include a subset of measurements where both complex and magnitude-only data from the same acquisition were used to fit T1 maps, thus the total number of unique acquisitions is lower than the numbers reported above. The datasets were collected on three MRI manufacturers (Siemens, GE, Philips) and were acquired at 3.0 T, except for one dataset acquired at 350 mT. To showcase the heterogeneity of the independently-implemented submissions, Figure 2 displays six T1 maps of the phantoms submitted to the challenge.
</p>

<p style="text-align:justify;">
Of these datasets, several submissions went beyond the minimum acquisition and acquired additional datasets using the NIST phantom, such as a traveling phantom (7 scanners), scan-rescan , same-day rescans on two MRIs, short TR vs long TR, and 4 point TI vs 14 point TI. For humans, one site acquired 13 subjects on three scanners (two manufacturers), one site acquired 6 subjects , and one site acquired a subject using two different head coils (20 channels vs. 64 channels).
</p>

<b style="text-align:justify;">
Figure 2. Example T1 maps that were submitted. Note the differences in acquisitions (e.g. FOV (top middle), orientation (bottom right, k-space pattern (top left and right) and resulting artifacts in the T1 maps (e.g. ghosting (bottom left), ringing (bottom middle), noise profiles (top left and bottom right), deformation/slice mispositioning (top right)) resulting from the independently-implemented acquisition protocols.
</b>

In [ ]:
# Imports

from pathlib import Path
import pandas as pd
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import Video
import glob
from analysis.src.plots import *
from analysis.make_pooled_datasets import *

# Configurations
configFile = Path('analysis/configs/3T_NIST_T1maps.json')
data_folder_name = 'analysis/3T_NIST_T1maps'
output_gif_folder = Path("analysis/plots/01-wholedataset_gif_NIST/")
output_gif_name = 'NIST.gif'

# Download datasets
if not Path(data_folder_name).exists():
    make_pooled_dataset(configFile, data_folder_name)

# Plot figure
fig = plt.figure()

#Set background color to white
fig.set_facecolor("w")

# Set scalebar range
vmin = 0
vmax = 3000

with open(configFile) as json_file:
    configJson = json.load(json_file)

ims = []

for dataset_name in configJson:
    for key1 in configJson[dataset_name]:
        if key1 == 'datasets':
            for key2 in configJson[dataset_name][key1]:         
                
                # Load T1 image data
                t1_file = configJson[dataset_name]['datasets'][key2]['imagePath']
                t1 = nib.load(Path(data_folder_name) / t1_file)
                t1_volume = t1.get_fdata() 

                # Handle 2D vs 3D volume case
                dims = t1_volume.shape
                if (len(dims) == 2) or (np.min(dims) == 1):
                    im = plt.imshow(np.rot90(t1_volume), cmap=parula_map, animated=True, vmin = vmin, vmax = vmax)
                else:
                    index_smallest_dim = np.argmin(dims)
                    numberOfSlices = dims[index_smallest_dim]
                    midSlice = int(np.round(numberOfSlices/2))

                    if index_smallest_dim == 0:
                        im = plt.imshow(np.rot90(np.squeeze(t1_volume[midSlice,:,:])), cmap=parula_map, animated=True, vmin = vmin, vmax = vmax)
                    elif index_smallest_dim == 1:
                        im = plt.imshow(np.rot90(np.squeeze(t1_volume[:,midSlice,:])), cmap=parula_map, animated=True, vmin = vmin, vmax = vmax)
                    elif index_smallest_dim == 2:
                        im = plt.imshow(np.rot90(np.squeeze(t1_volume[:,:,midSlice])), cmap=parula_map, animated=True, vmin = vmin, vmax = vmax)
                
                # Add filename text to image
                ttl = plt.text(0, 280, Path(t1_file).name, fontsize=6, horizontalalignment='left', verticalalignment='top')
                
                ims.append([im, ttl])

# Save all images to GIF
plt.colorbar()
plt.clim(vmin,vmax)

ani = animation.ArtistAnimation(fig, ims, interval=500, blit=False,repeat_delay=1000)

output_gif_folder.mkdir(parents=True, exist_ok=True)
ani.save(output_gif_folder / output_gif_name, dpi=100, writer='imagemagick')

plt.close()


In [ ]:
show_gif(output_gif_folder / output_gif_name)